<a href="https://colab.research.google.com/github/kimjc95/computational-chemistry/blob/main/Ligand_Docking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Conda 설치
#@markdown 런타임이 종료되었다가 다시 연결되는데, 당황하지 마시고 이어서 실행하시면 됩니다.

# Colab에서 Conda 실행하기 위해 준비
!pip install -q condacolab
import condacolab
condacolab.install()
!rm -rf /usr/local/conda-meta/pinned

In [ ]:
#@title Dependencies 설치
#@markdown (1분 가량 소요)
#@markdown 이틈에 Receptor PDB 파일과 Ligand MOL 파일을 업로드 해주세요.

!git clone -q --depth=1 https://github.com/QVina/qvina.git
!chmod -R 755 /content/qvina
!mamba install -q -c conda-forge openbabel numpy scipy rdkit parmed prody biopython pdbfixer nglview
!pip3 install -q meeko py3Dmol pdb4amber

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Lipinski import RotatableBondSmarts
from IPython.display import Image
from pdbfixer import PDBFixer
from openmm.app import PDBFile
from Bio.PDB import *
from openbabel import pybel
import os
import locale
import numpy as np
import nglview as nv
from google.colab import output
output.enable_custom_widget_manager()


In [ ]:
#@title **1. Ligand.pdbqt 준비**
#@markdown Ligand MOL 파일의 이름을 입력해주세요.

#@markdown 구조를 확인하고 전하나 결합차수가 맞는지 확인해주세요.

#@markdown 그림에 나온 원자번호는 0부터 셉니다.

ligandFile = "FhbK.mol" #@param {type:"string"}

IPythonConsole.molSize = 1000,750
IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.drawOptions.addBondIndices = False

mol = Chem.MolFromMolFile(ligandFile)
Chem.SanitizeMol(mol)
Chem.rdCoordGen.AddCoords(mol)
mol

In [ ]:
# @markdown 수소를 자동으로 더합니다.
mol_wH = Chem.rdmolops.AddHs(mol, explicitOnly=False, addCoords=True)
mol_wH

In [ ]:
#@markdown 회전 가능한 공유결합을 붉은색으로 표시해줍니다.

d = Chem.Draw.rdMolDraw2D.MolDraw2DCairo(1000, 750)

n_rot_bonds = Chem.rdMolDescriptors.CalcNumRotatableBonds(mol_wH)
n_amide_bonds = Chem.rdMolDescriptors.CalcNumAmideBonds(mol_wH)
print("Number of rotatable bonds : "+str(n_rot_bonds))
print("Number of amide bonds : "+str(n_amide_bonds))

rot_atom_pairs = mol_wH.GetSubstructMatches(RotatableBondSmarts)
rot_bonds = list(set([mol_wH.GetBondBetweenAtoms(*ap).GetIdx() for ap in rot_atom_pairs]))

Chem.Draw.rdMolDraw2D.PrepareAndDrawMolecule(d, mol_wH, legend='Rotatable bonds', highlightBonds=rot_bonds)
d.WriteDrawingText("rotatable_bond_highlight.png")
Image("rotatable_bond_highlight.png")

In [ ]:
# @markdown 이제 원자가 아닌 결합에 번호를 붙여 표시해드립니다.

# @markdown 잘 보고 회전을 고정하고 싶은 결합 번호를 0 1 2 이런 식으로 입력해주세요.
IPythonConsole.drawOptions.addAtomIndices = False
IPythonConsole.drawOptions.addBondIndices = True

fix_bonds = "9 42" #@param {type:"string"}
fix_bonds_idx = [int(s) for s in fix_bonds.split(' ')]
mol_wH

In [ ]:
# @markdown 고정된 결합은 파랑색으로 표시하였습니다.

# @markdown 수정이 필요하시면 다시 위의 셀을 실행해주세요.

bond_colors = {}

for bond in Chem.rdchem.Mol.GetBonds(mol_wH):
    if bond.GetIdx() in fix_bonds_idx:
        bond_colors[bond.GetIdx()] = (0.0,0.0,0.8)
    elif bond in rot_bonds:
        bond_colors[bond.GetIdx()] = (0.8,0.0,0.0)

d2 = Chem.Draw.rdMolDraw2D.MolDraw2DCairo(1000, 750)

Chem.Draw.rdMolDraw2D.PrepareAndDrawMolecule(d2, mol_wH, legend='Rotatable bonds in Red, Fixed bonds in Blue',
                                            highlightBonds=rot_bonds, highlightBondColors=bond_colors)

d2.WriteDrawingText("fixed_bond_highlight.png")
Image("fixed_bond_highlight.png")

In [ ]:
# @markdown 리간드 3차원 구조를 최적화할 ForceField를 선택해주세요.

FFtype = "UFF" #@param ['MMFF','UFF']

# @markdown 최적화된 구조는 ligand.sdf에 저장됩니다.

Chem.AllChem.EmbedMolecule(mol_wH)

if FFtype == 'UFF':
    Chem.AllChem.UFFOptimizeMolecule(mol_wH, maxIters=200)
else:
    Chem.AllChem.MMFFOptimizeMolecule(mol_wH, maxIters=200)
mblock = Chem.MolToMolBlock(mol_wH)
writer = Chem.SDWriter('ligand.sdf')
writer.write(mol_wH, confId=0)


view = nv.show_rdkit(mol_wH)
view

In [ ]:
# @markdown prepare_ligand.py로 ligand.pdbqt 파일을 준비합니다.

SMILES = Chem.rdmolfiles.MolToSmiles(mol_wH)
rotatable = ""

for i in fix_bonds_idx:
    bond = mol_wH.GetBondWithIdx(i)
    a1 = bond.GetBeginAtom().GetIdx()+1
    a2 = bond.GetEndAtom().GetIdx()+1
    rotatable = rotatable + ' -r "'+SMILES+'" -b '+str(a1)+" "+str(a2)

if os.system('mk_prepare_ligand.py -i ligand.sdf -o ligand.pdbqt'+rotatable) == 0:
    print("ligand.pdbqt successfully generated.")

In [ ]:
# @title **2. Receptor.pdbqt 준비**
# @markdown Receptor PDB 파일명을 입력해주세요.

receptorFile = "MaPylRS.pdb" # @param {type:"string"}

# @markdown 수소를 더해줄 기준 pH를 설정하세요.
pH = 7.0 #@param {type:"slider", min:0.0, max:14.0, step:0.1}
# @markdown 고쳐진 PDB 파일은 receptor-final.pdb로 저장됩니다.

fixer = PDBFixer(filename=receptorFile)
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(pH)

with open('receptor-fixed.pdb', 'w') as f:
    PDBFile.writeFile(fixer.topology, fixer.positions, f)

os.system('pdb4amber -i receptor-fixed.pdb -o receptorwithH.pdb -p --most-populous')

parser = PDBParser()
s = parser.get_structure('receptor', 'receptorwithH.pdb')

for chain in s.get_chains():
    for atom in chain[1].get_atoms():
        if atom.get_name() == 'H':
            setattr(atom, "id", 'H1')
            setattr(atom, "name", 'H1')
            setattr(atom, "fullname", ' H1  ')
            print(atom.id)

com = s.center_of_mass()
s.transform(np.array([[1,0,0],[0,1,0],[0,0,1]]), tran=-1*com)


io = PDBIO()
io.set_structure(s)
io.save("receptor-final.pdb")

with open("receptor-final.pdb") as f:
    view = nv.show_file(f, ext="pdb")

view


In [11]:
# @markdown Grid Box center 좌표 (in Angstrom)를 입력하세요. 혹은 위 위젯에서 원자를 클릭하세요.

center_x = "-5" #@param {type:"string"}
center_y = "0" #@param {type:"string"}
center_z = "5" #@param {type:"string"}

if center_x == "" or center_y == "" or center_z == "":
    if view.picked == {}:
        raise ValueError("좌표가 입력되지 않았습니다.")
    else:
        center_x = view.picked["atom1"]["x"]
        center_y = view.picked["atom1"]["y"]
        center_z = view.picked["atom1"]["z"]
else:
    center_x = float(center_x)
    center_y = float(center_y)
    center_z = float(center_z)

print(f"Box Center is {center_x:.3f}, {center_y:.3f}, {center_z:.3f}")


Box Center is -5.000, 0.000, 5.000


In [ ]:
# @markdown Grid Box 길이, 폭, 높이를 입력하세요.
len_x = "40" #@param {type:"string"}
len_y = "45" #@param {type:"string"}
len_z = "35" #@param {type:"string"}

with open('receptor-final.pdb', 'r') as f:
    view2 = nv.show_file(f, ext='pdb')


# @markdown 이 셀을 실행 후 아래 위젯에서 박스 크기와 위치를 확인하세요.

# @markdown center 좌표나 box 크기를 바꾼 후 이 셀을 다시 실행해 확인하세요.

# @markdown 빨강이 x축, 초록이 y축, 파랑이 z축입니다.

len_x = float(len_x)
len_y = float(len_y)
len_z = float(len_z)

corner1 = [center_x+len_x/2, center_y+len_y/2, center_z+len_z/2]
corner2 = [center_x-len_x/2, center_y+len_y/2, center_z+len_z/2]
corner3 = [center_x-len_x/2, center_y-len_y/2, center_z+len_z/2]
corner4 = [center_x+len_x/2, center_y-len_y/2, center_z+len_z/2]
corner5 = [center_x+len_x/2, center_y+len_y/2, center_z-len_z/2]
corner6 = [center_x-len_x/2, center_y+len_y/2, center_z-len_z/2]
corner7 = [center_x-len_x/2, center_y-len_y/2, center_z-len_z/2]
corner8 = [center_x+len_x/2, center_y-len_y/2, center_z-len_z/2]

view2.shape.add_cylinder(corner1, corner2, [1,1,1], [0.1])
view2.shape.add_cylinder(corner2, corner3, [1,1,1], [0.1])
view2.shape.add_cylinder(corner3, corner4, [1,1,1], [0.1])
view2.shape.add_cylinder(corner4, corner1, [1,1,1], [0.1])
view2.shape.add_cylinder(corner1, corner5, [1,1,1], [0.1])
view2.shape.add_cylinder(corner2, corner6, [1,1,1], [0.1])
view2.shape.add_cylinder(corner3, corner7, [0,0,1], [0.2])
view2.shape.add_cylinder(corner4, corner8, [1,1,1], [0.1])
view2.shape.add_cylinder(corner5, corner6, [1,1,1], [0.1])
view2.shape.add_cylinder(corner6, corner7, [0,1,0], [0.2])
view2.shape.add_cylinder(corner7, corner8, [1,0,0], [0.2])
view2.shape.add_cylinder(corner8, corner5, [1,1,1], [0.1])

view2

In [ ]:
# @markdown 지정하고 싶은 Flexible residue들을 쉼표로 구분하여 입력하세요.

# @markdown "Chain ID : Residue name : Residue number" 식으로 공백없이 써주세요.
flexible_residues = "" # @param {type:"string"}

fr = flexible_residues.split(',')

flex_res = ""

if fr != [""]:
    for res in fr:
        flex_res = flex_res + " -f "+res

os.system('mk_prepare_receptor.py --pdb receptor-final.pdb -o receptor.pdbqt --box_size '+str(len_x)+' '+str(len_y)+' '+str(len_z)+' --box_center '+str(center_x)+' '+str(center_y)+' '+str(center_z)+flex_res)

In [ ]:
locale.getpreferredencoding = lambda: "UTF-8"

# @title **3. QuickVina2를 이용한 도킹**
output = "ligand_out.pdbqt"
# @markdown Exhaustiveness 값을 설정하세요. 클 수록 오래 걸립니다.
exhaustiveness = 8 #@param {type:"slider", min:8, max:512, step:8}
# @markdown 도킹 로그는 docking_log.txt에, 결과는 ligand_out.pdbqt에 저장됩니다.

if fr != [""]:
    flex_rec = " --flex receptor_flex.pdbqt"
else:
    flex_rec = ""

with open('config', 'w') as f:
    f.write("receptor = receptor.pdbqt\n")
    f.write("ligand = ligand.pdbqt\n")
    if fr != [""]:
        f.write("flex = receptor_flex.pdbqt\n")
    f.write("center_x = "+str(center_x)+'\n')
    f.write("center_y = "+str(center_x)+'\n')
    f.write("center_z = "+str(center_x)+'\n')
    f.write("size_x = "+str(len_x)+'\n')
    f.write("size_y = "+str(len_y)+'\n')
    f.write("size_z = "+str(len_z)+'\n')
    f.write("out = "+output+'\n')
    f.write("log = docking_log.txt\n")
    f.write("exhaustiveness = "+str(exhaustiveness))

!/content/qvina/bin/qvina2.1 --config config

ms = pybel.readfile('pdbqt', 'ligand_out.pdbqt')

for i, pose in enumerate(ms):
    pose.write("pdb", "docked_pose_"+str(i+1)+'.pdb')

In [ ]:
# @title **4. 결과 분석**
# @markdown 보고 싶은 pose를 선택하세요.
pose = 1 # @param [1,2,3,4,5,6,7,8,9]

view4 = nv.NGLWidget()
view4.add_structure(nv.BiopythonStructure(s))
view4.add_surface(lowResolution= True, smooth=1, opacity=0.4)
view4.add_structure(nv.FileStructure('docked_pose_'+str(pose)+'.pdb'))
view4